<a href="https://colab.research.google.com/github/FabriDeCastelli/ML-Regression-Assignment/blob/main/deliverable/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
Student: Fabrizio De Castelli

--- 
# IMPORTANT: all the submitted code should be in 2 cells
1) How you trained, evaluated and saved your model
2) How to load your model from a file, load the data and evaluate the model. Cell 2) should be running independently (even if cell 1 is not run)

In [61]:
# Import libraries
import io
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt

# Load data 
url = 'https://drive.switch.ch/index.php/s/TeDwnbYsBKRuJjv/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# Alternatively yo can load the data from file
# data_path = '../data/data.npz' # path to the .npz file storing the data
# data = np.load(data_path)

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x

# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y


# T1
print('-------- T1 ---------')

from sklearn.model_selection import train_test_split 

# Build design matrix for the task
ones = np.ones(shape=(x.shape[0], 1))
sin_x2 = np.sin(x[:, 1]).reshape(-1,1)
x1_times_x2 = np.multiply(x[:, 0], x[:, 1]).reshape(-1,1)
X = np.hstack((ones, x, sin_x2, x1_times_x2)) 

# Generate training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=0)

# Library for Linear Regression
from sklearn.linear_model import LinearRegression

# Init the model
lr = LinearRegression(fit_intercept=False)  


# Estimate parameters on the training set
lr.fit(X_train, y_train)
theta_hat = lr.coef_

### Linear Regression ###
from sklearn.metrics import mean_squared_error 

# Generate prediction on training set and evaluate performance with MSE
train_prediction = lr.predict(X_train)
train_performance = mean_squared_error(train_prediction, y_train)

# Generate prediction on test set and evaluate performance with MSE
test_prediction = lr.predict(X_test)
test_performance = mean_squared_error(test_prediction, y_test)

print('theta_hat = {}'.format(theta_hat))

print('MSE on training set: {}'.format(train_performance))
print('MSE on test set: {}'.format(test_performance))

# T2
print('-------- T2 ---------')

from sklearn.preprocessing import PolynomialFeatures

degree = 11
pol_feat = PolynomialFeatures(degree=degree, include_bias=False) 

x_train, x_test = train_test_split(x, train_size=0.7, shuffle=True, random_state=0)

Xpol_train = pol_feat.fit_transform(x_train)
Xpol_test = pol_feat.fit_transform(x_test)

lr = LinearRegression(fit_intercept = True)
lr.fit(Xpol_train, y_train)

train_prediction = lr.predict(Xpol_train)
train_performance = mean_squared_error(train_prediction, y_train)

test_prediction = lr.predict(Xpol_test)
test_performance = mean_squared_error(test_prediction, y_test)

print('MSE on training set: {}'.format(train_performance))
print('MSE on test set: {}'.format(test_performance))

# T3 (Bonus)

print('-------- T3 ---------')

from keras.models import Sequential
from keras.layers import Dense
from keras import losses
from tensorflow import keras
import pickle

def baseline_model():
  network = Sequential()
  network.add(Dense(20, activation="relu"))
  network.add(Dense(11, activation="tanh"))
  network.add(Dense(1))
  network.compile(optimizer="sgd", loss="mean_squared_error")
  network.fit(X_train, y_train, epochs = 1200, verbose = 0)
  return network

network = baseline_model()

pickle.dump(network, open('my_model.pickle', 'wb'))

test_prediction = network.predict(X_test)
test_performance = mean_squared_error(test_prediction, y_test)

print('MSE on test set: {}'.format(test_performance))


-------- T1 ---------
theta_hat = [ 1.31635295 -0.05128958 -0.57659976  0.42026517  0.03686637]
MSE on training set: 0.696251517689963
MSE on test set: 0.7516362518990539
-------- T2 ---------
MSE on training set: 0.019345074371081006
MSE on test set: 0.07060475599724211
-------- T3 ---------
19/19 [==============================] - 0s 1ms/step
MSE on test set: 0.014943057416662828


# Example on how to use baseline model:

In [ ]:
# Import libraries
import joblib
import io
import requests
import numpy as np

def evaluate_predictions(y_true, y_pred):
    """
    Evaluates the mean squared error between the values in y_true and the values
    in y_pred.
    ### YOU CAN NOT EDIT THIS FUNCTION ###
    :param y_true: Numpy array, the true target values from the test set;
    :param y_pred: Numpy array, the values predicted by your model.
    :return: float, the mean squared error between the two arrays.
    """
    assert y_true.shape == y_pred.shape
    return ((y_true - y_pred) ** 2).mean()


def load_model(filename):
    """
    Loads a Scikit-learn model saved with joblib.dump.
    This is just an example, you can write your own function to load the model.
    Some examples can be found in src/utils.py.
    :param filename: string, path to the file storing the model.
    :return: the model.
    """
    model = joblib.load(filename)

    return model

# Load the data
# This will be replaced with our private test data when grading the assignment

# Load data from url
url = 'https://drive.switch.ch/index.php/s/TeDwnbYsBKRuJjv/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# Alternatively yo can load the data from file
# data_path = '../data/data.npz'
# data = np.load(data_path)

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# Load the trained model
baseline_model_path = './baseline_model.pickle'
baseline_model = load_model(baseline_model_path)

# Change input
x = ...

# Predict on the given samples
y_pred = baseline_model.predict(x)

############################################################################
# STOP EDITABLE SECTION: do not modify anything below this point.
############################################################################

# Evaluate the prediction using MSE
mse = evaluate_predictions(y_pred, y)
print(f'MSE on whole dataset: {mse}')

# NOTE: NOW THIS CELL IS NOT WORKING SINCE YOU NEED TO CHANGE THE INPUT.
# DO IT AND EVERYTHING RUNS SMOOTH
